### 1.1 Get the data from wikipedia

Installing the libraries

In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import lxml.html as lh
import pandas as pd

Scrape Table Cells

In [12]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #Create a handle, page, to handle the contents of the website
page = requests.get(url) #Store the contents of the website under doc
doc = lh.fromstring(page.content) #Parse data that are stored between <tr>..</tr> of HTML
tr_els = doc.xpath('//tr')

In [13]:
#Check the length of the first 10 rows
[len(T) for T in tr_els[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Parse Table Header

In [14]:
tr_els = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_els[0]:
    i+=1
    name=t.text_content()
    print (i,':','"',name,'"')
    col.append((name,[]))

1 : " Postcode "
2 : " Borough "
3 : " Neighbourhood
 "


Creating Pandas DataFrame

Each header is appended to a tuple along with an empty list.

In [15]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_els)):
    #T is our j'th row
    T=tr_els[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Just to be sure, let’s check the length of each column. Ideally, they should all be the same

In [16]:
[len(C) for (title,C) in col]

[287, 287, 287]

Now we are ready to create the DataFrame:

In [17]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [18]:
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Delete "\n" in our dataframe

In [20]:
df = df.replace(r'\n','', regex=True) #Delete /n in all columns
df.columns = df.columns.str.replace(r'\n','') #Delete /n in header
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### 1.2 Data preparation according to the tusk

Drop all raws with Borough = 'Not assigned'

In [21]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index() #Reset index 
del df['index'] #delete additional index column
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


If Neighbourhood = 'Not assigned' replace it with 'Borough' value

In [22]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Group the repeating Post code

In [23]:
df = df.groupby(['Postcode','Borough'], sort=False).agg( ','.join)
df = df.reset_index()
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [24]:
df.shape

(103, 3)

Now data is ready for the next tusk!